In [1]:
import numpy as np
from scipy.stats import gmean
import pandas as pd
from bokeh.plotting import figure, show, output_notebook
import itertools

In [2]:
def draw(seed, step):
    np.random.seed(seed)
    df = pd.DataFrame([],
                      index=[np.random.choice(np.arange(3), p=[1 / 2, 1 / 3, 1 / 6])
                             for _ in range(step)],
                      columns=[0, 1, 2, 3])
    df.loc[0, 0] = 3
    df.loc[1, 1] = 2
    df.loc[2, 2] = 6
    df.loc[:, 3] = 1
    df.fillna(0, inplace=True)
    return df

In [3]:
def make_draws(step, trials):
    data = [draw(i, step) for i in range(trials)]
    return data

In [4]:
def get_return(df, weight):
    return_df = df * np.array(weight)
    return np.exp(np.log(return_df.sum(axis=1)).cumsum()).values

In [5]:
def get_return_gmean(draws, weight):
    data = np.array([get_return(d, weight) for d in draws])
    return gmean(data[:, -1])

### ルーレットを100x1000回まわす

In [6]:
draws = make_draws(100, 1000)

DataFrameに当たりと、リターンが格納される  
0: A  
1: B  
2: C  
3: 手元の資金

In [7]:
draws[0].head(10)

,0,1,2,3
1,0,2,0,1
1,0,2,0,1
1,0,2,0,1
1,0,2,0,1
0,3,0,0,1
1,0,2,0,1
0,3,0,0,1
2,0,0,6,1
2,0,0,6,1
0,3,0,0,1


In [8]:
draws[-1].head(10)

,0,1,2,3
1,0,2,0,1
1,0,2,0,1
0,3,0,0,1
1,0,2,0,1
0,3,0,0,1
0,3,0,0,1
0,3,0,0,1
1,0,2,0,1
1,0,2,0,1
1,0,2,0,1


## 投資比率を変更してAだけに賭ける

### 賭ける比率

In [9]:
bet_a_pct = np.arange(0.01, 1, 0.01)
bet_a_weight = pd.DataFrame(bet_a_pct, index=bet_a_pct)
bet_a_weight[1] = 0
bet_a_weight[2] = 0
bet_a_weight[3] = 1 - bet_a_weight[0]

In [10]:
bet_a_weight.head()

,0,1,2,3
0.01,0.01,0,0,0.99
0.02,0.02,0,0,0.98
0.03,0.03,0,0,0.97
0.04,0.04,0,0,0.96
0.05,0.05,0,0,0.95


### 賭けた比率とルーレットの結果からリターンを算出

In [11]:
bet_a_result = bet_a_weight.apply(lambda x: get_return_gmean(draws, x), axis=1)

#### 100回再投資した場合のリターンの幾何平均
降順でソート

In [12]:
bet_a_result.sort_values(ascending=False).head(5)

0.25    371.250735
0.26    368.357790
0.24    367.572751
0.27    359.043412
0.23    357.514395
dtype: float64

#### 可視化

In [13]:
output_notebook()
p = figure(y_axis_type='log', plot_width=400, plot_height=300)
p.line(bet_a_pct, bet_a_result)
p.title.text = '100回繰り返し賭けた場合を1000回施行した幾何平均'
p.yaxis.axis_label = 'リターン'
p.xaxis.axis_label = 'Aに賭ける比率'
show(p)

Loading BokehJS ...

## 投資比率を組み合わせ

A、B、C、手元に残す比率を18分割し、全ての組み合わせを作成

In [14]:
def make_ix(data):
    pattern_dict = {0: '0', 1: '1/18', 2: '1/9', 3: '1/6', 4: '4/18', 5: '5/18',
                    6: '1/3', 7: '7/18', 8: '4/9', 9: '1/2', 10: '5/9',
                    11: '11/18', 12: '2/3', 13: '13/18', 14: '7/9',
                    15: '5/9', 16: '8/9', 17: '17/18', 18: '1'}
    return '{} {} {} {}'.format(*[pattern_dict[x] for x in data])


strike = range(19)
bet_all = pd.DataFrame(list(itertools.product(strike, strike, strike, strike)))
bet_all = bet_all.loc[bet_all.sum(axis=1) == 18]
bet_all.index = bet_all.apply(make_ix, axis=1)
bet_all = bet_all / 18
bet_all.head()

,0,1,2,3
0 0 0 1,0.0,0.0,0.000000,1.000000
0 0 1/18 17/18,0.0,0.0,0.055556,0.944444
0 0 1/9 8/9,0.0,0.0,0.111111,0.888889
0 0 1/6 5/9,0.0,0.0,0.166667,0.833333
0 0 4/18 7/9,0.0,0.0,0.222222,0.777778


### 上記組み合わせから結果を取得

In [15]:
bet_all_result = bet_all.apply(lambda x: get_return_gmean(draws, x), axis=1)

/home/driller/py36/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until
/home/driller/py36/lib/python3.6/site-packages/scipy/stats/stats.py:314: RuntimeWarning: divide by zero encountered in log
  log_a = np.log(a)


降順でソート

In [16]:
bet_all_result.sort_values(ascending=False).head(10)

7/18 1/6 1/9 1/3        877.027503
5/18 0 1/18 2/3         877.027503
1/2 1/3 1/6 0           877.027503
4/9 5/18 1/6 1/9        765.457578
1/3 1/9 1/9 4/9         765.457578
1/3 1/18 1/18 5/9       763.024419
4/9 4/18 1/9 4/18       763.024419
5/18 1/18 1/18 11/18    738.691044
7/18 4/18 1/9 5/18      738.691044
1/2 5/18 1/6 1/18       732.048813
dtype: float64